In [1]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor, LGBMClassifier, plot_tree
from sklearn.metrics import r2_score, f1_score, plot_confusion_matrix
import pickle5 as pickle
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import modelling
import numpy as np
from nilearn import plotting, datasets, surface  

Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.


In [ ]:
m = pickle.load(open("/home/brunovieira/Tamires_Experiments/Outputs/exp1/finalized_model.sav",'rb'))

In [ ]:
fsaverage_mesh_right = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/rh.pial'
fsaverage_thickness_right = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/rh.thickness'
fsaverage_annot_right = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/label/rh.Eco.annot'
fsaverage_sulc_right = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/rh.avg_sulc' 
fsaverage_mesh_left = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/lh.pial'
fsaverage_thickness_left = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/lh.thickness'
fsaverage_annot_left = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/label/lh.Eco.annot'
fsaverage_sulc_left = '/home/brunovieira/Tamires_Experiments/Bases_de_Dados/nki_data/fsaverage/surf/lh.avg_sulc'

In [ ]:
score_segmented = m.test('score_segmented')
our_econo = [2,3,4,5,6,7,8,9,10,11,12,13,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]
all_econo = pd.Series(list(surface.load_surf_data(fsaverage_annot_right))).unique()

extras = list(set(all_econo) - set(our_econo))
extras = [[a,0.0001] for a in extras]

r2_right = score_segmented[38:]
r2_right = [[our_econo[i],b] for i, [a,b] in enumerate(r2_right)]
r2_right = r2_right + extras
a_right = pd.DataFrame(surface.load_surf_data(fsaverage_annot_right))
b_right = pd.DataFrame(r2_right).set_index(0)
result_right = a_right.join(b_right, on=0).rename(columns={0:'econo',1:'r2'})

r2_left = score_segmented[:38]
r2_left = [[our_econo[i],b] for i, [a,b] in enumerate(r2_left)]
r2_left = r2_left + extras
a_left = pd.DataFrame(surface.load_surf_data(fsaverage_annot_left))
b_left = pd.DataFrame(r2_left).set_index(0)
result_left = a_left.join(b_left, on=0).rename(columns={0:'econo',1:'r2'})

In [ ]:
Age_Stable = pd.read_csv('/home/brunovieira/Tamires_Experiments/Bases_de_Dados/Age_Stable.csv', index_col = 0)
Age_Stable_left['age_stable'] = [np.nan if a<40 or a>85 else a for a in Age_Stable_left.age_stable ]
Age_Stable_left = Age_Stable[Age_Stable.hemisferio==1]
Age_Stable_right = Age_Stable[Age_Stable.hemisferio==0]
result_right = result_right.join(Age_Stable_right.set_index('econo'), on='econo')
result_left = result_left.join(Age_Stable_left.set_index('econo'), on='econo')

In [2]:
Age_Stable = pd.read_csv('/home/brunovieira/Tamires_Experiments/Bases_de_Dados/Age_Stable.csv', index_col = 0)


In [5]:
Age_Stable.mean()

econo           22.477273
age_stable      53.852273
hemisferio       0.500000
bigest_slope    -0.030217
dtype: float64

In [ ]:
result_left

In [ ]:
plt.scatter(result_left.r2, result_left.age_stable)

In [ ]:
plt.hist(result_left.age_stable, bins=40)
plt.show()

In [ ]:
for view in ['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']:
    fig = plotting.plot_surf_stat_map(surf_mesh = fsaverage_mesh_right, 
                                      stat_map = np.array(result_right.r2), 
                                      bg_map = fsaverage_sulc_right,
                                      hemi='right', 
                                      view=view,
                                      title = view +' view of right hemi ',
                                      colorbar=True,
                                      cmap = 'winter',
                                      #threshold=0.01,
                                      symmetric_cbar = False,
                                      bg_on_data=True,
                                      engine='plotly') 
fig.show() 

for view in ['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']:
    fig = plotting.plot_surf_stat_map(surf_mesh = fsaverage_mesh_left, 
                                      stat_map = np.array(result_left.r2), 
                                      bg_map = fsaverage_sulc_left,
                                      hemi='left', 
                                      view=view,
                                      title = view +' view of left hemi ',
                                      colorbar=True,
                                      cmap = 'winter',
                                      #threshold=0.01,
                                      symmetric_cbar = False,
                                      bg_on_data=True,
                                      engine='plotly') 
fig.show() 

In [ ]:
for view in ['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']:
    fig = plotting.plot_surf_stat_map(surf_mesh = fsaverage_mesh_right, 
                                      stat_map = np.array(result_right.bigest_slope), 
                                      bg_map = fsaverage_sulc_right,
                                      hemi='right', 
                                      view=view,
                                      title = view +' view of right hemi ',
                                      colorbar=True,
                                      cmap = 'winter',
                                      #threshold=20,
                                      symmetric_cbar = False,
                                      bg_on_data=True,
                                      engine='plotly') 
fig.show() 

for view in ['lateral', 'medial', 'dorsal', 'ventral', 'anterior', 'posterior']:
    fig = plotting.plot_surf_stat_map(surf_mesh = fsaverage_mesh_left, 
                                      stat_map = np.array(result_left.bigest_slope), 
                                      bg_map = fsaverage_sulc_left,
                                      hemi='left', 
                                      view=view,
                                      title = view +' view of left hemi ',
                                      colorbar=True,
                                      cmap = 'winter',
                                      #threshold=20,
                                      symmetric_cbar = False,
                                      bg_on_data=True,
                                      engine='plotly') 
fig.show() 